# **Preprocesamiento de datos**

In [23]:
# Librerias
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf

print("GPU disponible:", tf.config.list_physical_devices('GPU'))


In [31]:
# Lectura del dataset original
file = r"C:\Users\Hp\Downloads\accepted_2007_to_2018Q4.csv\accepted_2007_to_2018Q4.csv"
df = pd.read_csv(file, low_memory=False)
print("Dimensiones del DataFrame original:", df.shape)

Dimensiones del DataFrame original: (2260701, 151)


## **Reducción del dataset a la variable objetivo**
1. Definición de la variable objetivo `default`. 
2. Se filtran las filas donde `loan_status` es _Charged Off_ o _Fully Paid_.
3. Aseguramos que solo estas dos categorías estén presentes en loan_status
4. Se fija: (1) _Default_ (0) _Fully Paid_
5. Eliminamos la columna `loan_status` y creamos una nueva columna `default`


In [25]:
# Definición de la variable objetivo `default`

df = df[df['loan_status'].isin(['Charged Off', 'Fully Paid'])].copy()
df['default'] = df['loan_status'].apply(lambda x: 1 if x == 'Charged Off' else 0)
df.drop(columns=['loan_status'], inplace=True)
print("Dimensiones del DataFrame con la variable original", df.shape)

Dimensiones del DataFrame con la variable original (1345310, 151)


In [26]:
print("Información del DataFrame:")
df.info()

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
Index: 1345310 entries, 0 to 2260697
Columns: 151 entries, id to default
dtypes: float64(113), int64(1), object(37)
memory usage: 1.5+ GB


## **Tratamiento de variables**

### _Transformación y agregación_

#### **Codificación Dummy**

Codificación de variables tipo objeto de dos o tres categorias como variables dummy. Se eliminan del dataset las variables tipo objeto.

Conversion a variables dummy:
|   Variable    |   Categorías  |   Codificación |
|---------------|---------------|----------------|
| `debt_settlement_flag`  | 'Y' - 'N'     | 1 - 0          |
| `term`        | '36 months' - '60 months'  | 0-1 |
| `initial_list_status` | 'w' - 'f'          | 0-1 |
| `application_type`    | 'Individual' - 'Joint App' | 0-1          |
| `disbursement_method` | 'Cash' - 'DirectPay'` | 0-1                |
| `verification_status` | 'Not Verified' - 'Source Verified' - 'Verified' | 0-1-2                   |

In [ ]:
#Variables binarias con Y/N o y/n
df['had_settlement'] = df['debt_settlement_flag'].apply(lambda x: 1 if x=='Y' else 0)

# Diccionarios de mapeo para variables binarias categoricas
binary_maps = {
    'term': {' 36 months': 0, ' 60 months': 1},
    'initial_list_status': {'w': 0, 'f': 1},
    'application_type': {'Individual': 0, 'Joint App': 1},
    'disbursement_method': {'Cash': 0, 'DirectPay': 1},
    'verification_status': {'Not Verified': 0, 'Source Verified': 1,'Verified': 2},
}
# Aplicar el mapeo
for col, mapping in binary_maps.items():
    df[col + '_num'] = df[col].map(mapping)

In [6]:
# Eliminar columnas originales
df.drop(columns=['pymnt_plan', 'hardship_flag', 'debt_settlement_flag'], inplace=True)
df.drop(columns=binary_maps.keys(), inplace=True)

#### **Variables de inversor**
Las variables '_inv' representan un valor dado por la perspectiva de inversores. Se cambia por la diferencia entre el valor original y el sugerido.

In [7]:
# Lista de columnas que terminan en _inv
inv_cols = [col for col in df.columns if col.endswith('_inv')]

# Eliminarlas
df.drop(columns=inv_cols, inplace=True)


#### **Variables _low y _high**
Representan los limites entre rangos de un cliente. Se toma la media entre los valores.

In [8]:
# Buscar todas las columnas que terminan en _low
low_cols = [col for col in df.columns if col.endswith('_low')]

for low_col in low_cols:
    high_col = low_col.replace('_low', '_high')
    if high_col in df.columns:
        # Crear columna promedio
        mean_col = low_col.replace('_low', '_mean')
        df[mean_col] = np.round((df[low_col] + df[high_col]) / 2, 2)
        df.drop(columns=[low_col, high_col], inplace=True)

# Verificar las nuevas columnas
print([col for col in df.columns if col.endswith('_mean')])


['fico_range_mean', 'last_fico_range_mean', 'sec_app_fico_range_mean']


#### Variable DTI

In [9]:
# Reemplazar -1 y 999 por NaN
df['dti'] = df['dti'].replace([-1, 999], np.nan)
df.loc[df['dti'] > 100, 'dti'] = np.nan
df['dti'].isna().value_counts()


dti
False    1344401
True         909
Name: count, dtype: int64

#### **Variables tipo objeto**

In [10]:
# Extraer el número de sub_grade y obtener el número de sub_grade
df['sub_grade_num'] = df['sub_grade'].str[1].astype(int)
df.drop(columns=['sub_grade'], inplace=True)

#### **Tipo fecha**
Transformación de variables tipo MM-YYYY en String, a dos variables tipo DateTime '_year' y '_month'

In [11]:
# Convertir columnas de fecha y extraer año y mes
date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']
for col in date_cols:
    df[col] = pd.to_datetime(df[col], format="%b-%Y", errors='coerce')
# generar año y mes en un solo paso y concatenar
new_features = pd.concat(
    [df[col].dt.year.rename(col + "_year") for col in date_cols] +
    [df[col].dt.month.rename(col + "_month") for col in date_cols],
    axis=1
)

# concatenar con el dataframe original y eliminar las columnas originales
df = pd.concat([df.drop(columns=date_cols), new_features], axis=1)

#### **Variables de frecuencia de tiempo**
Mapeo y transformación a variable tipo numérica

In [12]:
# Creamos un diccionario de mapeo
emp_map = {
    '10+ years': 10,
    '9 years': 9,
    '8 years': 8,
    '7 years': 7,
    '6 years': 6,
    '5 years': 5,
    '4 years': 4,
    '3 years': 3,
    '2 years': 2,
    '1 year': 1,
    '< 1 year': 0,
    np.nan: -1,  # Asignamos NaN a 'n/a'
}
df['emp_length_num'] = df['emp_length'].map(emp_map)
# Eliminamos la columna original
df.drop(columns=['emp_length'], inplace=True)

#### creacion de variables representativas

In [13]:
# Agregacion y eliminacion 
df['payment_effiency'] = df['total_rec_prncp']/df['total_pymnt']
df['loan_gap'] = df['loan_amnt'] - df['funded_amnt']


df.drop(columns=['collection_recovery_fee','loan_amnt',
                 'open_acc',
                 'avg_cur_bal', 'pub_rec', 'mths_since_rcnt_il'
                 ], inplace=True)

### _Eliminación de variables identificadoras_
Se eliminan las variables que permiten diferenciar entre cada registro como 'id', 'url', 'member:id'

In [14]:
df = df.drop(columns=['id','url','emp_title','member_id', 'title', 'zip_code','addr_state',
                      'member_id', 'policy_code'])


-----------------

### _Valores nulos_

In [15]:
# Conteo de variables con valores faltantes
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]
print("Variables con valores faltantes:", len(missing))

Variables con valores faltantes: 100


#### **Variables con más del 70% de datos perdidos**
Se eliminaran por ser poco informativas y difíciles de imputar.\
Variables como 'pymnt', 'hardship', 'settlement', representadas por el booleano anteriormente codificado.

In [16]:
# Missing 70%
missing = df.isnull().sum().sort_values(ascending=False)
umbral = 0.7 * len(df)
faltan_70= missing[missing > umbral]
print("Columnas con más del 70% de valores faltantes:", len(faltan_70))

df.drop(columns=faltan_70.index, inplace=True)

Columnas con más del 70% de valores faltantes: 40


#### **Variables meses desde**

Las variables que indican meses desde alguna acción.
Si existen NA si no tienen acción se llenan como -1 para indicar que no paso o no es el caso

In [17]:
# Variable since_last
since_last = [col for col in df.columns if 'since' in col.lower()]
print("Variables que contienen 'since':", since_last)
# Imputar valores faltantes en las variables 'since_last' con -1
df[since_last] = df[since_last].fillna(-1)


Variables que contienen 'since': ['mths_since_last_delinq', 'mths_since_recent_bc', 'mths_since_recent_inq', 'mths_since_recent_revol_delinq']


#### **Variables con más del 50% de datos perdidos**
Se agrupan variables por tipo, codificando como flags y un resumen por categorías.

In [18]:
# Missing 50%
missing = df.isnull().sum().sort_values(ascending=False)
umbral = 0.5* len(df)
faltan_50= missing[missing > umbral]
print("Columnas con más del 50% de valores faltantes:", len(faltan_50))

Columnas con más del 50% de valores faltantes: 13


#### **Agrupación de variables por características**

In [ ]:
#gruparon en los scores
delinq_cols = ['num_tl_30dpd','num_accts_ever_120_pd']
credit_hist_cols = ['mo_sin_old_il_acct','mo_sin_old_rev_tl_op','mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl',
                    'num_actv_rev_tl','num_actv_bc_tl','num_op_rev_tl','num_il_tl','num_bc_tl',
                    'num_rev_accts','num_rev_tl_bal_gt_0','tot_cur_bal','total_il_high_credit_limit',
                    'total_rev_hi_lim']
balances_cols = ['percent_bc_gt_75','bc_open_to_buy','total_bal_ex_mort','total_bc_limit']
recent_activity_cols = ['last_pymnt_d_month','last_pymnt_d_year',
                        'last_credit_pull_d_month','last_credit_pull_d_year','inq_last_6mths']
credit_util_cols = ['il_util']


open_acc_cols = ['open_acc_6m', 'open_il_12m', 'open_il_24m', 'open_rv_24m']
inq_cols = ['inq_last_12m', 'inq_fi']
balance_cols = ['total_cu_tl']

# Eliminar columnas originales de scores
df.drop(columns=delinq_cols + credit_hist_cols + balances_cols + recent_activity_cols+ credit_util_cols + open_acc_cols + inq_cols + balance_cols, inplace=True)


#### **Variables restantes con valores nulos**

In [20]:
# Conteo de variables con valores faltantes
missing = df.isnull().sum().sort_values(ascending=False)
missing = missing[missing > 0]
print("Variables con valores faltantes:", len(missing))
table = pd.DataFrame(missing, columns=['missing_count'])
table['percentage'] = (table['missing_count'] / len(df)) * 100
table.sort_values(by='percentage', ascending=False, inplace=True)
print(table)

Variables con valores faltantes: 23
                            missing_count  percentage
all_util                           807765   60.043038
open_rv_12m                        807712   60.039099
open_act_il                        807712   60.039099
total_bal_il                       807712   60.039099
max_bal_bc                         807712   60.039099
num_tl_120dpd_2m                   117401    8.726688
pct_tl_nvr_dlq                      67681    5.030885
tot_coll_amt                        67527    5.019438
num_tl_90g_dpd_24m                  67527    5.019438
num_tl_op_past_12m                  67527    5.019438
tot_hi_cred_lim                     67527    5.019438
bc_util                             61912    4.602062
num_bc_sats                         55841    4.150791
num_sats                            55841    4.150791
mort_acc                            47281    3.514506
acc_open_past_24mths                47281    3.514506
dti                                   909    0

----------------

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1345310 entries, 0 to 2260697
Data columns (total 64 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   funded_amnt                     1345310 non-null  float64
 1   int_rate                        1345310 non-null  float64
 2   installment                     1345310 non-null  float64
 3   grade                           1345310 non-null  object 
 4   home_ownership                  1345310 non-null  object 
 5   annual_inc                      1345310 non-null  float64
 6   purpose                         1345310 non-null  object 
 7   dti                             1344401 non-null  float64
 8   delinq_2yrs                     1345310 non-null  float64
 9   mths_since_last_delinq          1345310 non-null  float64
 10  revol_bal                       1345310 non-null  float64
 11  revol_util                      1344453 non-null  float64
 12  total

In [22]:
df.to_csv("data_lend.csv", index=False)
